In [84]:
import sklearn.decomposition
import pandas as pd
import math
import string
import re
import numpy as np
import nltk 
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from nltk.corpus import stopwords
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from __future__ import division



In [85]:
tesis = pd.read_csv('tesis.csv',sep=';',header=0)

In [86]:
def limpiar(texto):
    list_to_clean = ['\x0c','\n']
    for c in list_to_clean:
        texto = texto.replace(c,' ')
    clean_text = re.sub('[^a-zA-Z]+',' ',texto)    
    return clean_text.lower()

tesis['ts_clean'] = tesis['abstract'].apply(lambda x:limpiar(x))

In [87]:
def tokenize(text):
    text = re.sub("[^a-zA-Z]"," ",text)
    text = re.sub(r'\b\w{1,2}\b',' ',text)
    text = re.sub(r'http[\bs]*',' ',text)
    text = re.sub(r'telefonos','telefono',text)
    tokens = nltk.word_tokenize(text)
    return tokens

In [88]:
#tf_vectorizer = CountVectorizer(analyzer='word', tokenizer=tokenize, max_features = 2, lowercase=True
                               #,stop_words=nltk.corpus.stopwords.words('spanish'))
#tf = tf_vectorizer.fit_transform(tesis['tes_clean'])

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   stop_words=nltk.corpus.stopwords.words('spanish'))
tfidf = tfidf_vectorizer.fit_transform(tesis['ts_clean'])

In [89]:
corpus_features = tfidf.toarray()
palabras = tfidf_vectorizer.get_feature_names()
print palabras

[u'about', u'academic', u'acceptable', u'access', u'according', u'account', u'achieve', u'achieved', u'acquisition', u'activities', u'activity', u'adaptations', u'adapted', u'addition', u'address', u'adequate', u'administration', u'administrative', u'advantage', u'advertising', u'after', u'against', u'aim', u'aimed', u'aims', u'all', u'allowing', u'allows', u'also', u'alternatives', u'among', u'amount', u'an', u'analysis', u'analyzed', u'analyzing', u'annual', u'any', u'appears', u'application', u'approximately', u'are', u'area', u'areas', u'arises', u'around', u'as', u'aspects', u'assessment', u'assist', u'assistance', u'at', u'attractive', u'bank', u'based', u'basis', u'be', u'because', u'been', u'before', u'beginning', u'benefit', u'benefits', u'besides', u'between', u'bicycle', u'bike', u'born', u'both', u'business', u'but', u'by', u'cameras', u'can', u'capital', u'capitalist', u'care', u'careers', u'carried', u'carry', u'cash', u'center', u'centers', u'cfn', u'chain', u'change', u

In [90]:
x_train, x_test, y_train, y_test = train_test_split(corpus_features,tesis['sentimiento']
                                                    ,train_size= .85, random_state=40)
print len(x_train),len(x_test)
print len(x_train),len(y_test)

31 6
31 6


In [91]:
from sklearn.linear_model import LogisticRegression
lm = LogisticRegression()
lm = lm.fit(X=x_train,y=y_train)

In [92]:
from sklearn.metrics import classification_report
y_pred = lm.predict(x_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

         -1       0.33      1.00      0.50         2
          1       0.00      0.00      0.00         4

avg / total       0.11      0.33      0.17         6



In [93]:
tesis['abstract'][1] 

"BikeRun S.A. emerges as a need or lack which is increasing in the city of Guayaquil, the pace and conditions of work, somehow justify that people does not motivate to exercise and this may lead to stress because of the physical and mental exhaustion after a hard day's work. Thus arises the need for business that allows to solve this problem to a large extent, giving you a choice of quality of life. The proposal of this project is to provide a product with opportunities and alternatives to people who can devote the time necessary to exercise using a device built into a bicycle, that beyond scroll and enjoy a relaxing, healthy and non-polluting will allow people monitor, keep track of your travel, speed and travel time, calories, heart rate. The city of Guayaquil is a very broad market and business approach is aimed at all persons of different ages and social classes. BikeRun to the market-access strategy is to create in people the need and the importance of having this device and which

In [94]:
dataset = fetch_20newsgroups(subset='all', shuffle=True, random_state=42)
labels = dataset.target
true_k = np.unique(labels).shape[0]
svd = TruncatedSVD(true_k)
lsa = make_pipeline(svd, Normalizer(copy=False))

X = lsa.fit_transform(tfidf)

km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100)
km.fit(X)

KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=20, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)